In [163]:
import pandas as pd
from my_utils2 import *
import tqdm from tqdm

SyntaxError: invalid syntax (2281441428.py, line 3)

### 닉네임과 tag를 통해 매치데이터를 임의의 숫자만큼 가져와서 그사람이 플레이한 챔피언들의 데이터프레임 만들기
### 결과컬럼 - [챔피언이름 , 전체게임횟수, 승,패,승률,kills,deaths,assists,kda]

In [ ]:
def get_data(nickname, tag, num, queue):
    url = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{nickname}/{tag}?api_key={api_key}'
    res = requests.get(url).json()
    puuid = res['puuid']
    
    url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue={queue}&type=ranked&start=0&count={num}&api_key={api_key}'
    res2 = requests.get(url2).json()
    
    
    
    

In [417]:
name = 'Tagdx'
tag = '3597'
api_key = 'RGAPI-798eb514-99d5-4a07-a165-d21c43381dc0'
num = 50
queue = 420

In [341]:
url = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}?api_key={api_key}'

In [343]:
res = requests.get(url).json()

In [347]:
puuid = res['puuid']

In [348]:
puuid

'N5LHp-boX7Mf0dUUJqRrcdrstVmsyAMnQN2RCd_wq0jzlGJMd5KRij3zdpToMLHLr12lOh_5ALusWw'

In [360]:
url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue={queue}&type=ranked&start=0&count={num}&api_key={api_key}'

In [456]:
df_create = []
for match_id in tqdm(requests.get(url2).json()):
    url3 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    matches = requests.get(url3).json()
    tmp = []
    try:
        for match in matches['info']['participants']:
            if match['riotIdGameName']== name and match['riotIdTagline'] == tag:
                tmp.append(match['championName'])
                tmp.append(match['win'])
                tmp.append(match['kills'])
                tmp.append(match['deaths'])
                tmp.append(match['assists'])
        df_create.append(tmp)
    except:
        try:
            for match in matches['info']['participants']:
                if match['riotIdName']== name and match['riotIdTagline'] == tag:
                    tmp.append(match['championName'])
                    tmp.append(match['win'])
                    tmp.append(match['kills'])
                    tmp.append(match['deaths'])
                    tmp.append(match['assists'])
            df_create.append(tmp)
        except:
            continue

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:13<00:00,  1.46s/it]


In [465]:
df = pd.DataFrame(df_create, columns = ['championName', 'win', 'kills', 'deaths', 'assists'])

In [466]:
cnt_df = df[['championName', 'win']].groupby('championName').count().rename(columns = {'win' : 'cnt'})

In [468]:
sum_df = df.groupby('championName').sum()

In [470]:
result_df = pd.merge(sum_df, cnt_df, on = 'championName')

In [472]:
result_df['lose'] = result_df.cnt - result_df.win

In [474]:
result_df['win_rate'] = round(result_df.win / result_df.cnt, 2)*100

In [476]:
result_df['kda'] = round((result_df.kills + result_df.assists) / result_df.deaths, 2)

In [478]:
result_df[['cnt', 'win', 'lose', 'win_rate', 'kills', 'deaths', 'assists', 'kda']].sort_values(by = ['cnt', 'win_rate'], ascending=False)

,cnt,win,lose,win_rate,kills,deaths,assists,kda
championName,,,,,,,,
Nidalee,17,7,10,41.0,139,98,124,2.68
Graves,6,4,2,67.0,42,36,47,2.47
LeeSin,6,1,5,17.0,29,43,45,1.72
Kindred,5,2,3,40.0,32,38,33,1.71
Ekko,3,2,1,67.0,22,14,24,3.29
Evelynn,3,1,2,33.0,18,16,12,1.88
Fiora,2,1,1,50.0,7,10,5,1.20
Viego,2,1,1,50.0,9,13,16,1.92
Karthus,1,1,0,100.0,6,5,15,4.20
